In [11]:
import openai
from vllm import LLM, SamplingParams
import huggingface_hub

In [15]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('HUG_API_KEY')

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import huggingface_hub
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('HUG_API_KEY')
huggingface_hub.login(token=api_key)

# Transformers로 모델 로드
tokenizer = AutoTokenizer.from_pretrained("Min-kyu/PJA_LLM_MODEL_8bit")
model = AutoModelForCausalLM.from_pretrained(
    "Min-kyu/PJA_LLM_MODEL_8bit",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 사용 예시
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 테스트
result = generate_text("안녕하세요")
print(result)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [28]:
from fastapi import FastAPI
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import uvicorn
from pydantic import BaseModel
import huggingface_hub
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('HUG_API_KEY')
huggingface_hub.login(token=api_key)

app = FastAPI()

# 모델 로드 (서버 시작시 한번만)
tokenizer = AutoTokenizer.from_pretrained("Min-kyu/PJA_LLM_MODEL_8bit")
model = AutoModelForCausalLM.from_pretrained(
    "Min-kyu/PJA_LLM_MODEL_8bit",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

class GenerateRequest(BaseModel):
    prompt: str
    max_length: int = 100
    temperature: float = 0.7

@app.post("/generate")
async def generate_text(request: GenerateRequest):
    inputs = tokenizer(request.prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=request.max_length,
            do_sample=True,
            temperature=request.temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"generated_text": result}

@app.get("/health")
async def health_check():
    return {"status": "healthy"}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: asyncio.run() cannot be called from a running event loop